# Plantilla: Proyecto
## Tema 2: Clasificación de barrios en función de su tipología.
Documento plantilla para el proyecto con los imports de las librerias más importantes.

In [1]:
import nltk                      # Natural Language Toolkit
import scrapy                    # Web scraping   
import requests                  # HTTP requests
import selenium                  # Web automation
import numpy as np               # Numerical computing
import pandas as pd              # Data manipulation
import seaborn as sns            # Data visualization
import tensorflow as tf          # Machine learning
import matplotlib.pyplot as plt  # Data visualization
import os                        # Operating system

!pip install openpyxl
import openpyxl                  # Excel files

# Automatización en la recogida de variables (dataset Barris)

In [2]:
# tengo un .csv en el path ./data/barrios/barrios_codigos.csv. Me interesan las columnas Nombre, Codigo distrito y Codigo barrio, quédate con esas.

idBarrios = pd.read_csv('./data/barrios/barrios_codigos.csv', delimiter=';')
idBarrios = idBarrios[['Nombre', 'Codigo distrito', 'Codigo barrio']]

barrios_dict = {}

for index, row in idBarrios.iterrows():
    nombre = row['Nombre']
    codigo_distrito = row['Codigo distrito']
    codigo_barrio = row['Codigo barrio']
    
    barrio_dict = {
        'Codigo distrito': codigo_distrito,
        'Codigo barrio': codigo_barrio
    }
    
    barrios_dict[nombre] = barrio_dict

In [3]:
def get_barrio_name(codigo_distrito, codigo_barrio):
    for nombre, barrio_dict in barrios_dict.items():
        if barrio_dict['Codigo distrito'] == codigo_distrito and barrio_dict['Codigo barrio'] == codigo_barrio:
            return nombre

In [4]:
sheets = pd.ExcelFile('./data/barrios/2024/Districte_01_Barri_1.xlsx').sheet_names
print(sheets)

['Padrón', 'MovPadrón', 'Censo', 'Vehículos', 'Catastro', 'ActEconom', 'Elecciones']


In [5]:
def find_string_in_df(df, string):
    for i in range(len(df)):
        if string in str(df.iloc[i, 0]):
            return i
    return None

def extract_df(df, row, skip, n = None):
    if n is None:
        # Encontrar n dinámicamente.
        last_row = row + skip
        while last_row < len(df) and not df.iloc[last_row].isna().all():
            last_row += 1
        n = last_row - row - skip
    extracted_df = df.iloc[row + skip:row + skip + n, :]
    
    # Verificar si TODAS las filas en cada columna son NA
    columns_to_drop = extracted_df.columns[extracted_df.isna().all()]
    
    # Eliminar solo las columnas donde todas las filas son NA
    extracted_df = extracted_df.drop(columns_to_drop, axis=1)
    
    return extracted_df

def extract_df_by_string(df, string, skip, n=None):
    row = find_string_in_df(df, string)
    return extract_df(df, row, skip, n)


In [6]:
def find_string_in_df(df, string):
    for i in range(len(df)):
        if string in str(df.iloc[i, 0]):
            return i
    return None

def find_skip(df, row):
    # Encontrar la primera fila que no esté completamente llena de NAs después de la fila especificada
    for i in range(row + 2, len(df)):
        if not df.iloc[i].isna().all():
            return i - row

    return 0  # Si todas las filas después de 'row' están completamente llenas de NAs, skip es 0

def extract_df(df, row, skip, n=None):
    if n is None:
        # Encontrar n dinámicamente.
        last_row = row + skip
        while last_row < len(df) and not df.iloc[last_row].isna().all():
            last_row += 1
        n = last_row - row - skip
    extracted_df = df.iloc[row + skip:row + skip + n, :]
    
    # Verificar si TODAS las filas en cada columna son NA
    columns_to_drop = extracted_df.columns[extracted_df.isna().all()]
    
    # Eliminar solo las columnas donde todas las filas son NA
    extracted_df = extracted_df.drop(columns_to_drop, axis=1)
    
    return extracted_df

def extract_df_by_string(df, string, n=None):
    row = find_string_in_df(df, string)
    skip = find_skip(df, row)
    return extract_df(df, row, skip, n)

In [13]:
barrioPadron = pd.read_excel('./data/barrios/2024/Districte_01_Barri_1.xlsx', sheet_name = 'Padrón')
barrioMovPadron = pd.read_excel('./data/barrios/2024/Districte_01_Barri_1.xlsx', sheet_name = 'MovPadrón')
barrioVehiculos = pd.read_excel('./data/barrios/2024/Districte_01_Barri_1.xlsx', sheet_name = 'Vehículos')
barrioCatastro = pd.read_excel('./data/barrios/2024/Districte_01_Barri_1.xlsx', sheet_name = 'Catastro')
barrioElecciones = pd.read_excel('./data/barrios/2024/Districte_01_Barri_1.xlsx', sheet_name = 'Elecciones')

In [8]:
dfSup = extract_df_by_string(barrioPadron, string='Superficie y densidad de población')
dfPobSexEdad = extract_df_by_string(barrioPadron, string='Población por sexo y edad (grandes grupos)', skip = 3, n = 4)
dfPobSexLugar = extract_df_by_string(barrioPadron, string='Población según lugar de nacimiento y sexo', skip = 3, n = 4)
dfPobExtCont = extract_df_by_string(barrioPadron, string='Población extranjera según nacionalidad, por contienentes', skip = 3, n = 4)
dfHojasFamComp = extract_df_by_string(barrioPadron, string='Número de hojas familiares según composición', skip = 3, n = 2)
dfIndicadoresDemograficosPadron = extract_df_by_string(barrioPadron, string='Indicadores demográficos', skip = 4, n = None)


dfMovPadron = extract_df_by_string(barrioMovPadron, string='Resumen de movimientos registrados en el Padrón Municipal', skip = 3, n = 2)
dfIndicadoresDemograficosMovPadron = extract_df_by_string(barrioMovPadron, string='Indicadores demográficos', skip = 4, n = None)



dfVehicTipo = extract_df_by_string(barrioVehiculos, string='Vehículos según tipo', skip = 3, n = None)
dfTurismPot = extract_df_by_string(barrioVehiculos, string='Turismos según potencia', skip = 3, n = None)



dfBienesInmAno = extract_df_by_string(barrioCatastro, string='Bienes Inmuebles según el año de antigüedad', skip = 3, n = 2)
dfBienesInmSup = extract_df_by_string(barrioCatastro, string='Bienes Inmuebles construidas después de 1800 según superficie construida', skip = 3, n = 2)

TypeError: extract_df_by_string() got an unexpected keyword argument 'skip'

In [14]:
# Pon todo lo mismo que antes pero quitando en todas el skip y el n en las llamadas.

dfSup = extract_df_by_string(barrioPadron, string='Superficie y densidad de población')
dfPobSexEdad = extract_df_by_string(barrioPadron, string='Población por sexo y edad (grandes grupos)')
dfPobSexLugar = extract_df_by_string(barrioPadron, string='Población según lugar de nacimiento y sexo')
dfPobExtCont = extract_df_by_string(barrioPadron, string='Población extranjera según nacionalidad, por contienentes')
dfHojasFamComp = extract_df_by_string(barrioPadron, string='Número de hojas familiares según composición')
dfIndicadoresDemograficosPadron = extract_df_by_string(barrioPadron, string='Indicadores demográficos')

dfMovPadron = extract_df_by_string(barrioMovPadron, string='Resumen de movimientos registrados en el Padrón Municipal')
dfIndicadoresDemograficosMovPadron = extract_df_by_string(barrioMovPadron, string='Indicadores demográficos')

dfVehicTipo = extract_df_by_string(barrioVehiculos, string='Vehículos según tipo')
dfTurismPot = extract_df_by_string(barrioVehiculos, string='Turismos según potencia')

dfBienesInmAno = extract_df_by_string(barrioCatastro, string='Bienes Inmuebles según el año de antigüedad')
dfBienesInmSup = extract_df_by_string(barrioCatastro, string='Bienes Inmuebles construidas después de 1800 según superficie construida')
dfBienesInmVal = extract_df_by_string(barrioCatastro, string='Bienes Inmuebles construidas después de 1800 según valor catastral')
dfBienesValMedio = extract_df_by_string(barrioCatastro, string='Valores catastrales medios')
dfSupAparc = extract_df_by_string(barrioCatastro, string='Superficie total de los aparcamientos')

dfElecGen = extract_df_by_string(barrioElecciones, string='Votos a candidaturas')

# Mete eso de arriba en una función que reciba un nombre del diccionario a crearse con todo ello, va a ser un diccionario de dataframes.
    


In [23]:
# Mete los dataframes de arriba en un diccionario de dataframes, donde la clave sea el nombre del dataframe y el valor sea el dataframe.

def extract_dataframes_from_excel(excel_path):
    barrioPadron = pd.read_excel(excel_path, sheet_name = 'Padrón')
    barrioMovPadron = pd.read_excel(excel_path, sheet_name = 'MovPadrón')
    barrioVehiculos = pd.read_excel(excel_path, sheet_name = 'Vehículos')
    barrioCatastro = pd.read_excel(excel_path, sheet_name = 'Catastro')
    barrioElecciones = pd.read_excel(excel_path, sheet_name = 'Elecciones')

    dfSup = extract_df_by_string(barrioPadron, string='Superficie y densidad de población')
    dfPobSexEdad = extract_df_by_string(barrioPadron, string='Población por sexo y edad (grandes grupos)')
    dfPobSexLugar = extract_df_by_string(barrioPadron, string='Población según lugar de nacimiento y sexo')
    dfPobExtCont = extract_df_by_string(barrioPadron, string='Población extranjera según nacionalidad, por contienentes')
    dfHojasFamComp = extract_df_by_string(barrioPadron, string='Número de hojas familiares según composición')
    dfIndicadoresDemograficosPadron = extract_df_by_string(barrioPadron, string='Indicadores demográficos')

    dfMovPadron = extract_df_by_string(barrioMovPadron, string='Resumen de movimientos registrados en el Padrón Municipal')
    dfIndicadoresDemograficosMovPadron = extract_df_by_string(barrioMovPadron, string='Indicadores demográficos')

    dfVehicTipo = extract_df_by_string(barrioVehiculos, string='Vehículos según tipo')
    dfTurismPot = extract_df_by_string(barrioVehiculos, string='Turismos según potencia')

    dfBienesInmAno = extract_df_by_string(barrioCatastro, string='Bienes Inmuebles según el año de antigüedad')
    dfBienesInmSup = extract_df_by_string(barrioCatastro, string='Bienes Inmuebles construidas después de 1800 según superficie construida')
    dfBienesInmVal = extract_df_by_string(barrioCatastro, string='Bienes Inmuebles construidas después de 1800 según valor catastral')
    dfBienesValMedio = extract_df_by_string(barrioCatastro, string='Valores catastrales medios')
    dfSupAparc = extract_df_by_string(barrioCatastro, string='Superficie total de los aparcamientos')
    
    dfElecGen = extract_df_by_string(barrioElecciones, string='Votos a candidaturas')
    
    dataframes = { 'dfSup': dfSup,
                     'dfPobSexEdad': dfPobSexEdad,
                     'dfPobSexLugar': dfPobSexLugar,
                     'dfPobExtCont': dfPobExtCont,
                     'dfHojasFamComp': dfHojasFamComp,
                     'dfIndicadoresDemograficosPadron': dfIndicadoresDemograficosPadron,
                     'dfMovPadron': dfMovPadron,
                     'dfIndicadoresDemograficosMovPadron': dfIndicadoresDemograficosMovPadron,
                     'dfVehicTipo': dfVehicTipo,
                     'dfTurismPot': dfTurismPot,
                     'dfBienesInmAno': dfBienesInmAno,
                     'dfBienesInmSup': dfBienesInmSup,
                     'dfBienesInmVal': dfBienesInmVal,
                     'dfBienesValMedio': dfBienesValMedio,
                     'dfSupAparc': dfSupAparc,
                     'dfElecGen': dfElecGen
                  }
    
    return dataframes

Seu_2024 = extract_dataframes_from_excel('./data/barrios/2024/Districte_01_Barri_1.xlsx')

In [26]:
Seu_2024['dfPobSexEdad']

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
26,NaN,Total,0-15,16-64,65 i més
27,Total,3151,350,2183,618
28,Homes,1514,177,1065,272
29,Dones,1637,173,1118,346
